In [1]:
import tensorflow as tf
import numpy as np
from tensorflow.examples.tutorials.mnist import input_data

In [2]:
mnist=input_data.read_data_sets('MNIST_data',one_hot=True)

Instructions for updating:
Please use alternatives such as official/mnist/dataset.py from tensorflow/models.
Instructions for updating:
Please write your own downloading logic.
Instructions for updating:
Please use tf.data to implement this functionality.
Extracting MNIST_data\train-images-idx3-ubyte.gz
Instructions for updating:
Please use tf.data to implement this functionality.
Extracting MNIST_data\train-labels-idx1-ubyte.gz
Instructions for updating:
Please use tf.one_hot on tensors.
Extracting MNIST_data\t10k-images-idx3-ubyte.gz
Extracting MNIST_data\t10k-labels-idx1-ubyte.gz
Instructions for updating:
Please use alternatives such as official/mnist/dataset.py from tensorflow/models.


In [3]:
mnist.train

In [4]:
lr=0.001
training_iters=1000
batch_size=64

In [5]:
n_inputs=28
n_steps=28
n_hidden_units=128
n_classes=10
NUM_LAYERS=2#lstm的层数

In [6]:
#dataset = tf.data.Dataset.from_tensor_slices((X,Y))
#dataset=dataset.shuffle(1000).repeat().batch(batch_size)#shuffle(2000).
#iterator=dataset.make_one_shot_iterator()
#next_element=iterator.get_next()

In [7]:
x=tf.placeholder(tf.float32,[None,n_steps,n_inputs])
y=tf.placeholder(tf.float32,[None,n_classes])

In [8]:
weights={
    'in':tf.Variable(tf.random_normal([n_inputs,n_hidden_units])),
    'out':tf.Variable(tf.random_normal([n_hidden_units,n_classes]))
}
biases={
    'in':tf.Variable(tf.constant(0.1,shape=[n_hidden_units])),
    'out':tf.Variable(tf.constant(0.1,shape=[n_classes,]))
}

In [9]:
def RNN(X,weights,biases):
    X=tf.reshape(X,[-1,n_inputs])
    X_in=tf.matmul(X,weights['in'])+biases['in']
    X_in=tf.reshape(X_in,[-1,n_steps,n_hidden_units]) 
    
    
##############    这个深层rnn的实现    ###############################   
    cell=tf.nn.rnn_cell.MultiRNNCell([
        tf.nn.rnn_cell.BasicLSTMCell(n_hidden_units)
        for _ in range(NUM_LAYERS)])
    outputs,states=tf.nn.dynamic_rnn(cell,X_in,dtype=tf.float32)
    results=tf.matmul(states[1].h,weights['out'])+biases['out']
    
    
##############    这是单层lstm的实现   #############################    
#     lstm_cell=tf.nn.rnn_cell.BasicLSTMCell(n_hidden_units,forget_bias=1.0,state_is_tuple=True)
#     _init_state=lstm_cell.zero_state(batch_size,dtype=tf.float32)
#     outputs,states=tf.nn.dynamic_rnn(lstm_cell,X_in,initial_state=_init_state,time_major=False)
#     results=tf.matmul(states[1],weights['out'])+biases['out']
    
    
    
    #outputs=tf.unpack(tf.transpose(outputs,[1,0,2]))
    #results=tf.matmul(outputs[-1],weights['out'])+biases['out']
    return results

In [10]:
XX=[]
YY=[]
np.size(XX)

0

In [11]:

    
step=0
while step*batch_size<training_iters:
    batch_xs,batch_ys=mnist.train.next_batch(batch_size)#mnist.train.next_batch(batch_size)
    batch_xs=batch_xs.reshape([batch_size,n_steps,n_inputs])
    if np.size(XX)==0:
        XX=batch_xs
        YY=batch_ys
    else:
        XX=np.vstack((XX,batch_xs))
        YY=np.vstack((YY,batch_ys))
    step+=1

In [12]:
dataset = tf.data.Dataset.from_tensor_slices((XX,YY))
dataset=dataset.shuffle(8000).batch(batch_size).repeat()#shuffle(2000).
iterator=dataset.make_one_shot_iterator()
next_element=iterator.get_next()

In [13]:
pred=RNN(x,weights,biases)
cost=tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits=pred,labels=y))
train_op = tf.train.AdamOptimizer(lr).minimize(cost)

correct_pred=tf.equal(tf.argmax(pred,1),tf.argmax(y,1))
accuracy=tf.reduce_mean(tf.cast(correct_pred,tf.float32))

init=tf.initialize_all_variables()
with tf.Session() as sess:
    sess.run(init)
    step=0
    while step<training_iters:#step*batch_size
        batch_xs,batch_ys=sess.run(next_element)#mnist.train.next_batch(batch_size)
        batch_xs=batch_xs.reshape([batch_size,n_steps,n_inputs])
        sess.run([train_op],feed_dict={
            x:batch_xs,
            y:batch_ys,
        })
        if step%20==0:
            print(sess.run(accuracy,feed_dict={
                x:batch_xs,
                y:batch_ys,
            }))
        step+=1

Instructions for updating:

Future major versions of TensorFlow will allow gradients to flow
into the labels input on backprop by default.

See @{tf.nn.softmax_cross_entropy_with_logits_v2}.

Instructions for updating:
Use `tf.global_variables_initializer` instead.
0.125
0.75
0.8125
0.9375
0.875
0.921875
0.953125
1.0
0.96875
1.0
1.0
0.984375
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
